<a href="https://colab.research.google.com/github/JHMamun/Performance_testing_NB_SVM/blob/main/Nb_Svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving emails.csv to emails.csv


In [ ]:
df = pd.read_csv('emails.csv')

In [ ]:
#drop duplicates
df.drop_duplicates(inplace= True)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def process_text(text):

  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)

  clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

  return clean_words

In [ ]:
df['text'].head().apply(process_text)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
messages_bow = CountVectorizer(analyzer= process_text).fit_transform(df['text'])

In [ ]:
#training 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(messages_bow, df['spam'], test_size = 0.20, random_state = 0)

In [ ]:
#NB classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train, y_train)

In [ ]:
#prediction
print(classifier.predict(X_train))
print(y_train.values)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train, pred))
print('Confusion Matrix: \n', confusion_matrix(y_train, pred))
print('Accuracy: ', accuracy_score(y_train, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

    accuracy                           1.00      4556
   macro avg       0.99      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556

Confusion Matrix: 
 [[3445   12]
 [   1 1098]]
Accuracy:  0.9971466198419666


In [ ]:
print(classifier.predict(X_test))
print(y_test.values)

[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(y_test, pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, pred))
print('Accuracy: ', accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139

Confusion Matrix: 
 [[862   8]
 [  1 268]]
Accuracy:  0.9920983318700615


In [ ]:
#svm method
from sklearn import feature_extraction, model_selection, metrics, svm

list_C = np.arange(100, 2000, 100)
score_train = np.zeros(len(list_C))
score_test = np.zeros(len(list_C))
recall_test = np.zeros(len(list_C))
precision_test= np.zeros(len(list_C))
count = 0

for C in list_C:
  svc = svm.SVC(C=C)
  svc.fit(X_train, y_train)
  score_train[count] = svc.score(X_train, y_train)
  score_test[count]= svc.score(X_test, y_test)
  recall_test[count] = metrics.recall_score(y_test, svc.predict(X_test))
  precision_test[count] = metrics.precision_score(y_test, svc.predict(X_test))
  count = count + 1

In [ ]:
#creating model
matrix = np.matrix(np.c_[list_C, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = ['C', 'Train Accuracy', 'Test Accuracy', 'Test Recall', 'Test Precision'])
models.head(n=5)

,C,Train Accuracy,Test Accuracy,Test Recall,Test Precision
0,100.0,1.0,0.974539,0.914498,0.976190
1,200.0,1.0,0.973661,0.914498,0.972332
2,300.0,1.0,0.973661,0.914498,0.972332
3,400.0,1.0,0.974539,0.918216,0.972441
4,500.0,1.0,0.973661,0.918216,0.968627


In [ ]:
best_index = models['Test Precision'].idxmax()
models.iloc[best_index, :]

C                 100.000000
Train Accuracy      1.000000
Test Accuracy       0.974539
Test Recall         0.914498
Test Precision      0.976190
Name: 0, dtype: float64

In [ ]:
best_index = models['Test Precision'].idxmax()
models.iloc[best_index, :]

C                 100.000000
Train Accuracy      1.000000
Test Accuracy       0.974539
Test Recall         0.914498
Test Precision      0.976190
Name: 0, dtype: float64

In [ ]:
models[models['Train Accuracy']==1].head(n=5)

,C,Train Accuracy,Test Accuracy,Test Recall,Test Precision
0,100.0,1.0,0.974539,0.914498,0.976190
1,200.0,1.0,0.973661,0.914498,0.972332
2,300.0,1.0,0.973661,0.914498,0.972332
3,400.0,1.0,0.974539,0.918216,0.972441
4,500.0,1.0,0.973661,0.918216,0.968627


In [ ]:
best_index = models[models['Train Accuracy']==1]['Test Accuracy'].idxmax()
svc = svm.SVC(C=list_C[best_index])
svc.fit(X_train, y_train)
models.iloc[best_index, :]

C                 100.000000
Train Accuracy      1.000000
Test Accuracy       0.974539
Test Recall         0.914498
Test Precision      0.976190
Name: 0, dtype: float64

In [ ]:
m_confusion_test = metrics.confusion_matrix(y_test, svc.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'], index = ['Actual 0', 'Actual 1'])
print('Confusion matrix of SVM')
print(m_confusion_test)

Confusion matrix of SVM
[[864   6]
 [ 23 246]]
